# **크롤링용 필요 모듈**

In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
from urllib.request import urlopen
from selenium.webdriver.common.keys import Keys
import time

# **축제 데이터 크롤링 기능 확인**
- 1월의 서울에서 열리는 축제 중 맨 첫번째 데이터 가져오기

In [175]:
url = "https://korean.visitkorea.or.kr/kfes/list/wntyFstvlList.do"
driver = webdriver.Chrome()
driver.get(url)

time.sleep(1)

site_month = driver.find_elements(By.CSS_SELECTOR, value = "select#searchDate option")
# 4부터 15까지가 실질적 월

site_month[3].click()

site_area = driver.find_elements(By.CSS_SELECTOR, value = "select#searchArea option")
# 1부터 17까지가 실질적 지역 

site_area[1].click()

site_search = driver.find_element(By.CSS_SELECTOR, value = "button#btnSearch")

site_search.click()

time.sleep(1)

site_festival = driver.find_elements(By.CSS_SELECTOR, value = "ul#fstvlList a")

site_festival[0].click()

time.sleep(1)

soup = BeautifulSoup(driver.page_source, "lxml")

# 날짜 정보
site_festival_date = soup.select("div.info_ico.data ~p")

# 위치 정보
site_festival_location = soup.select("div.info_ico.location ~p")

# 사진 정보
site_festival_image = soup.select_one("div.detail_img_box img").get("src", "")

## **이미지 파일 저장 가능 확인**

In [180]:
with open("test.png", "wb") as f:
    f.write(urlopen(site_festival_image).read())

# 이미지 파일 저장 성공

## **크롤링 데이터 정상 여부 확인**

In [183]:
site_festival_location[0].text

# 데이터 크롤링 정상 확인

'서울특별시 송파구 송파나루길 206 송파관광정보센터 \xa0'

# **반복문을 통한 크롤링 기능 확인**
- 뒤로 가기 기능을 통해 1~12월까지 각 지역의 축제들 데이터 가져오기 성공

In [189]:
# 초안
url = "https://korean.visitkorea.or.kr/kfes/list/wntyFstvlList.do"
driver = webdriver.Chrome()
driver.get(url)

time.sleep(1)

site_month = driver.find_elements(By.CSS_SELECTOR, value = "select#searchDate option")
site_month = site_month[3:16]
for i in range(len(site_month)):
    site_month[i].click()
    # 월 클릭
    site_area = driver.find_elements(By.CSS_SELECTOR, value = "select#searchArea option")
    site_area = site_area[1:18]
    for j in range(len(site_area)):
        site_area[j].click()
        # 지역 클릭
        site_search = driver.find_element(By.CSS_SELECTOR, value = "button#btnSearch")
        while True:
            site_search.click()
            # 검색버튼 클릭
            time.sleep(1)
            site_festival = driver.find_elements(By.CSS_SELECTOR, value = "ul#fstvlList a")
            for k in range(len(site_festival)):
                site_festival[k].click()
                time.sleep(1)
                soup = BeautifulSoup(driver.page_source, "lxml")
                site_festival_date = soup.select_one("div.info_ico.data ~p")
                site_festival_location = soup.select_one("div.info_ico.location ~p")
                if soup.select_one("div.detail_img_box img"):
                    site_festival_image = soup.select_one("div.detail_img_box img").get("src", "")
                else:
                    pass
                festival = {"date" : site_festival_date.string, "location" : site_festival_location.string, "image" : site_festival_image}
                driver.back()
            break

KeyboardInterrupt: 

In [191]:
festival

{'date': '2025.01.28\n                                            ~ 2025.01.30',
 'location': '서울특별시 종로구 삼일대로 464 (운니동)\xa0',
 'image': 'https://kfescdn.visitkorea.or.kr/kfes/upload/contents/db/1852e51d-5c6b-465b-ad7f-bcb72ad556a6_3.png'}

## **딕셔너리 형태로 집어넣기**

In [69]:
url = "https://korean.visitkorea.or.kr/kfes/list/wntyFstvlList.do"
driver = webdriver.Chrome()
driver.get(url)

time.sleep(1)

site_month = driver.find_elements(By.CSS_SELECTOR, value = "select#searchDate option")
site_month = site_month[3:5] # 수정 16 → 4
for i in range(len(site_month)):
    site_month[i].click()
    # 월 클릭
    month = site_month[i].text # 추가
    site_area = driver.find_elements(By.CSS_SELECTOR, value = "select#searchArea option")
    site_area = site_area[1:2] # 수정 18 → 2
    for j in range(len(site_area)):
        site_area[j].click()
        # 지역 클릭
        area = site_area[j].text # 추가
        site_search = driver.find_element(By.CSS_SELECTOR, value = "button#btnSearch")
        while True:
            site_search.click()
            # 검색 버튼 클릭
            time.sleep(1)
            site_festival = driver.find_elements(By.CSS_SELECTOR, value = "ul#fstvlList a")
            for k in range(len(site_festival)):
                site_festival[k].click()
                # 검색된 축제 클릭
                time.sleep(1)

                # 축제 정보 가져오기 #
                soup = BeautifulSoup(driver.page_source, "lxml")
                site_festival_date = soup.select_one("div.info_ico.data ~p")
                # 시작 날짜, 종료날짜, 중간 날짜를 여기서 거를지에 대한 판단 필요 #
                
                site_festival_location = soup.select_one("div.info_ico.location ~p")
                if soup.select_one("div.detail_img_box img"):
                    festival_image = soup.select_one("div.detail_img_box img").get("src", "")
                else:
                    festival_image = soup.select_one("div.slick-slide img").get("src","")
                    
                # 축제 정보 할당 및 추가
                festivals.append({month : {area : {"date" : site_festival_date.string, "location" : site_festival_location.string, "image" : festival_image}}})
                driver.back()
            break

In [70]:
festivals

[{'01월': {'서울': {'date': '2024.10.25\n                                            ~ 2025.02.28',
    'location': '서울특별시 송파구 송파나루길 206 송파관광정보센터 \xa0',
    'image': 'https://kfescdn.visitkorea.or.kr/kfes/upload/contents/db/1852e51d-5c6b-465b-ad7f-bcb72ad556a6_3.png'}}},
 {'01월': {'서울': {'date': '2025.01.28\n                                            ~ 2025.01.30',
    'location': '서울특별시 종로구 삼일대로 464 (운니동)\xa0',
    'image': 'https://kfescdn.visitkorea.or.kr/kfes/upload/contents/db/abf1651d-0a57-44b0-b58d-06ed561e0d0a_3.jpg'}}},
 {'01월': {'서울': {'date': '2025.01.01\n                                            ~ 2025.01.01',
    'location': '서울특별시 광진구 광장동\xa0370-3 아차산 어울림광장',
    'image': 'https://kfescdn.visitkorea.or.kr/kfes/upload/contents/db/8a2de891-7ad4-422e-8c65-aca20a328f6b_3.jpg'}}},
 {'01월': {'서울': {'date': '2025.01.01\n                                            ~ 2025.01.01',
    'location': '서울특별시 종로구 청운동\xa0인왕산 청운공원, 청와대 분수광장 내 대고각(효자동 116-2)',
    'image': 'https://kfescdn.

## **리스트 형태로 집어넣기**

In [71]:
url = "https://korean.visitkorea.or.kr/kfes/list/wntyFstvlList.do"
driver = webdriver.Chrome()
driver.get(url)

time.sleep(1)

site_month = driver.find_elements(By.CSS_SELECTOR, value = "select#searchDate option")
site_month = site_month[3:4] # 수정 16 → 4
for i in range(len(site_month)):
    site_month[i].click()
    # 월 클릭
    month = site_month[i].text # 추가
    site_area = driver.find_elements(By.CSS_SELECTOR, value = "select#searchArea option")
    site_area = site_area[1:2] # 수정 18 → 2
    for j in range(len(site_area)):
        site_area[j].click()
        # 지역 클릭
        area = site_area[j].text # 추가
        site_search = driver.find_element(By.CSS_SELECTOR, value = "button#btnSearch")
        while True:
            site_search.click()
            # 검색 버튼 클릭
            time.sleep(1)
            site_festival = driver.find_elements(By.CSS_SELECTOR, value = "ul#fstvlList a")
            for k in range(len(site_festival)):
                site_festival[k].click()
                # 검색된 축제 클릭
                time.sleep(1)
                
                # 축제 정보 가져오기 #
                soup = BeautifulSoup(driver.page_source, "lxml")
                site_festival_date = soup.select_one("div.info_ico.data ~p")
                festival_date = site_festival_date.string
                # 시작 날짜, 종료날짜, 중간 날짜를 여기서 거를지에 대한 판단 필요 #
                
                site_festival_location = soup.select_one("div.info_ico.location ~p")
                festival_location = site_festival_location.string
                if soup.select_one("div.detail_img_box img"):
                    festival_image = soup.select_one("div.detail_img_box img").get("src", "")                    
                else:
                    festival_image = soup.select_one("div.slick-slide img").get("src","")
                    
                # 축제 정보 할당 및 추가
                festivals.append({month:{area:[festival_date, festival_location,festival_image]}})
                driver.back()
            break

In [72]:
festivals

[{'01월': {'서울': {'date': '2024.10.25\n                                            ~ 2025.02.28',
    'location': '서울특별시 송파구 송파나루길 206 송파관광정보센터 \xa0',
    'image': 'https://kfescdn.visitkorea.or.kr/kfes/upload/contents/db/1852e51d-5c6b-465b-ad7f-bcb72ad556a6_3.png'}}},
 {'01월': {'서울': {'date': '2025.01.28\n                                            ~ 2025.01.30',
    'location': '서울특별시 종로구 삼일대로 464 (운니동)\xa0',
    'image': 'https://kfescdn.visitkorea.or.kr/kfes/upload/contents/db/abf1651d-0a57-44b0-b58d-06ed561e0d0a_3.jpg'}}},
 {'01월': {'서울': {'date': '2025.01.01\n                                            ~ 2025.01.01',
    'location': '서울특별시 광진구 광장동\xa0370-3 아차산 어울림광장',
    'image': 'https://kfescdn.visitkorea.or.kr/kfes/upload/contents/db/8a2de891-7ad4-422e-8c65-aca20a328f6b_3.jpg'}}},
 {'01월': {'서울': {'date': '2025.01.01\n                                            ~ 2025.01.01',
    'location': '서울특별시 종로구 청운동\xa0인왕산 청운공원, 청와대 분수광장 내 대고각(효자동 116-2)',
    'image': 'https://kfescdn.

In [ ]:
# 데이터 전처리 과정 필요